In [1]:
# importaint libraries
import sys 
import amfe
import matplotlib.pyplot as plt
import numpy as np
import scipy
import copy
import pandas as pd
import os

directory = 'temp2'
system_path = os.path.join(directory, 'system.aft')
global_system = amfe.load_obj(system_path)

domain = global_system

domain.split_in_partitions()

problem_type = domain.problem_type
num_partitions = len(domain.groups)
partitions_list = np.arange(1,num_partitions+1)


master = amfe.Master()
master.subdomain_keys = partitions_list

domain_dict = {}
for sub_id in partitions_list:
    sub_i =  amfe.FETIsubdomain(domain.groups[sub_id])
    domain_dict[sub_id] = sub_i
    G_dict = sub_i.calc_G_dict()
    subdomain_interface_dofs_dict = sub_i.num_of_interface_dof_dict
    subdomain_null_space_size = sub_i.null_space_size
    master.append_G_dict(G_dict)
    master_dict = master.append_partition_dof_info_dicts(sub_id,subdomain_interface_dofs_dict,subdomain_null_space_size)
    sub_i.calc_dual_force_dict()
    master.append_null_space_force(sub_i.null_space_force,sub_id)


for sub_id in partitions_list:
    sub_i = domain_dict[sub_id]
    for nei_id in sub_i.neighbor_partitions:
        sub_j = domain_dict[nei_id]
        Gj = sub_j.G_dict[nei_id,sub_id]
        sub_i.append_neighbor_G_dict(nei_id,Gj)
        

for sub_id in partitions_list:
    sub_i = domain_dict[sub_id]
    GtG_row_dict = sub_i.calc_GtG_row()
    if GtG_row_dict:
        master.appendGtG_row(GtG_row_dict,sub_id)

master.build_local_to_global_mapping()
GtG = master.assemble_GGT()
G, e = master.assemble_G_and_e()

lambda_im = master.solve_lambda_im()

def action_of_global_F(lambda_im,master,partitions_list):
    for sub_id in partitions_list:
        sub_i = domain_dict[sub_id]
        local_h_dict = sub_i.apply_local_F(lambda_im,master.lambda_dict)
        master.append_h(local_h_dict)
        
    Fim = master.assemble_global_F_action() # Fpk = B*Kpinv*B'*pk
    return Fim

def assemble_global_d(master,partitions_list):
    for sub_id in partitions_list:
        sub_i = domain_dict[sub_id]
        master.append_d_hat(sub_i.dual_force_dict)

    d = master.assemble_global_d_hat() # dual force global assemble
    return d

d = assemble_global_d(master,partitions_list)
r0 = d -  action_of_global_F(lambda_im,master,partitions_list)
n_int = len(r0)

GGT_inv = np.linalg.inv(GtG)
F = lambda x : action_of_global_F(x,master,partitions_list)
P = lambda w : (np.eye(n_int) - G.T.dot(GGT_inv).dot(G)).dot(w)

lambda_ker, last_res, proj_r_hist, lambda_hist = amfe.PCGP(F,r0,P)
lambda_sol =  lambda_im + lambda_ker

dtil = d - F(lambda_sol)
GGT = GtG
alpha_pgc = np.linalg.solve(GGT,G.dot(dtil))


super_domain = amfe.SuperDomain(domain.groups,method='cholsps')
B = super_domain.assemble_global_B()
u = super_domain.eval_subdomain_displacement(lambda_sol,alpha_pgc)



Python was not able to load the fast fortran material routines.

Extract interface node from sub_1 and sub_2
Extract interface node from sub_1 and sub_3
Extract interface node from sub_1 and sub_4
Interface nodes from sub_2 and sub_1 already extracted
Extract interface node from sub_2 and sub_3
Extract interface node from sub_2 and sub_4
Interface nodes from sub_3 and sub_1 already extracted
Interface nodes from sub_3 and sub_2 already extracted
Extract interface node from sub_3 and sub_4
Interface nodes from sub_4 and sub_1 already extracted
Interface nodes from sub_4 and sub_2 already extracted
Interface nodes from sub_4 and sub_3 already extracted
Extract interface node from sub_1 and sub_2
Extract interface node from sub_1 and sub_3
Extract interface node from sub_1 and sub_4
Interface nodes from sub_2 and sub_1 already extracted
Extract interface node from sub_2 and sub_3
Extract interface node from sub_2 and sub_4
Interface nodes from sub_3 and sub_1 already extracted
Interface 

C:\Users\ge72tih\AppData\Local\Continuum\anaconda3\envs\py36\Lib\site-packages\scipy\sparse\compressed.py:742: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [2]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
def plot_static(scale=0,alpha_scale=1):
    u = super_domain.eval_subdomain_displacement(lambda_sol,alpha_scale*alpha_pgc)
    connectivity = {}
    nodes = {}
    fig, ax1=plt.subplots(1,figsize=(10, 5), dpi= 50, facecolor='w', edgecolor='k')
    for sub_key in super_domain.domains_key_list:
        sub = super_domain.get_feti_subdomains(sub_key)
        connectivity[sub_key] = sub.mesh.connectivity
        nodes[sub_key] = sub.mesh.nodes
        quad, ax = amfe.plotDeformQuadMesh(connectivity[sub_key],nodes[sub_key],u[sub_key],scale,ax1) 
          
    plt.show()
    return u

u = interact(plot_static,scale=(0,200,5),alpha_scale=(-1,1,0.1)) 

A Jupyter Widget

In [3]:
w, alpha_hat = master.solve_corse_grid(-dtil)

In [4]:
alpha_hat

array([  1.88550475e-02,   1.88567871e-02,   1.80558164e-12,
         1.88567871e-02,   1.88550475e-02,  -1.42679198e-03])

In [5]:
alpha_pgc

array([ -1.88550475e-02,  -1.88567871e-02,  -1.80559201e-12,
        -1.88567871e-02,  -1.88550475e-02,   1.42679198e-03])

In [6]:
dtil

array([ -1.88550475e-02,  -3.47932183e-06,  -1.88567871e-02,
        -3.47932187e-06,   1.69766805e-12,   3.27886378e-14,
         0.00000000e+00,   0.00000000e+00,   9.44619383e-13,
         1.10782277e-14,  -1.88567871e-02,   1.43027130e-03,
         1.88567871e-02,   3.47932190e-06,   0.00000000e+00,
         1.42679198e-03,   4.62650751e-15,   1.43375062e-03,
         1.56472058e-15,   1.43027130e-03,  -1.88567871e-02,
         1.43027130e-03,  -1.88550475e-02,   1.43027130e-03])

In [7]:
G

array([[  1.00000000e+00,  -2.00000000e+00,  -1.58383589e-15,
         -2.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          1.58383589e-15,   2.00000000e+00,  -0.00000000e+00,
         -0.00000000e+00,   1.58383589e-15,   2.00000000e+00,
          1.51461136e-15,   1.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [ -1.90700351e-15,   2.00000000e+00,   1.00000000e+00,
          2.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         -1.00000000e+00,  -2.00000000e+00,  -1.00000000e+00,
         -0.00000000e+00,  -1.00000000e+00,  -2.00000000e+00,
         -1.00000000e+00,  -1.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
      

In [8]:
lambda_sol

array([ -4.98855021e+07,   3.22662072e+05,  -2.50572490e+07,
         2.19265126e+05,   1.30223721e-03,   5.29262835e+05,
         0.00000000e+00,   0.00000000e+00,   7.25045745e-04,
        -6.03480978e+06,  -2.50572490e+07,   3.09997709e+05,
         2.50572490e+07,   3.09997709e+05,  -5.84173854e-06,
        -2.66989824e+04,   4.13900583e-06,   9.07325830e+04,
        -2.48918144e-06,   1.67895889e+05,  -2.50572490e+07,
        -2.19265126e+05,  -4.98855021e+07,  -3.22662072e+05])

In [9]:
F(lambda_sol)

array([  1.88550475e-02,   3.47932183e-06,   1.88567871e-02,
         3.47932187e-06,  -1.69766805e-12,  -3.27886378e-14,
         0.00000000e+00,   0.00000000e+00,  -9.44619383e-13,
        -1.10782277e-14,   1.88567871e-02,  -1.43027130e-03,
        -1.88567871e-02,  -3.47932190e-06,   0.00000000e+00,
        -1.42679198e-03,  -4.62650751e-15,  -1.43375062e-03,
        -1.56472058e-15,  -1.43027130e-03,   1.88567871e-02,
        -1.43027130e-03,   1.88550475e-02,  -1.43027130e-03])

In [10]:
d

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])